# パッケージ

In [1]:
import numpy as np
import pandas as pd

# 事前準備  
時系列株価データの取得

In [2]:
from datetime import date, timedelta
from pandas_datareader import data as pdr

In [3]:
# 期間設定
today = date.today()
startday = today - timedelta(days=20)
# 銘柄設定
ticker_list = ["IBM", "AAPL", "MSFT", "C", "T"]

In [4]:
# 取得
pd_data = pdr.DataReader(ticker_list, "stooq", startday)

In [5]:
# リターン
ret = pd_data['Close'].sort_index(ascending=True)
ret=ret.pct_change().drop(ret.index[0])

In [6]:
# 共分散行列
cvm = ret.cov()

In [7]:
# 株価
prc = pd_data['Close']['IBM'].sort_index(ascending=True)

In [9]:
prc.shape[0]

14

In [10]:
prc.index.searchsorted(prc.index + pd.Timedelta(days=1))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int64)

## 3.1 日次ボラティリティ推定

In [8]:
close=prc

In [9]:
# 2営業日前の日付を取得
df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
df0 = df0[df0>0]
df0 = pd.Series(close.index[df0-1], index=close.index[close.shape[0]-df0.shape[0]:])

# 2営業日前からのリターン系列
df0 = close.loc[df0.index] / close.loc[df0.values].values-1

# 指数平滑加重標準偏差
df0.ewm(span=100).std()

Date
2023-04-26         NaN
2023-04-27    0.003529
2023-04-28    0.002663
2023-05-01    0.004616
2023-05-02    0.007250
2023-05-03    0.011076
2023-05-04    0.012151
2023-05-05    0.011511
2023-05-08    0.010755
2023-05-09    0.011319
2023-05-10    0.010800
2023-05-11    0.010332
2023-05-12    0.010547
Name: IBM, dtype: float64

In [10]:
df0

Date
2023-04-26    0.003589
2023-04-27    0.008579
2023-04-28    0.004450
2023-05-01   -0.002531
2023-05-02   -0.009888
2023-05-03   -0.020937
2023-05-04   -0.020694
2023-05-05    0.001620
2023-05-08   -0.002022
2023-05-09   -0.020057
2023-05-10   -0.011183
2023-05-11   -0.002228
2023-05-12    0.006720
Name: IBM, dtype: float64

## 2.1 リスク分布RのPCAウェイト  
各主成分のリスク寄与が指定した配分となるような資産ウェイトを算出

In [36]:
# パラメータ設定（共分散行列、リスク水準）
cov = cvm
riskTarget = 1

In [22]:
# 固有値、固有ベクトル（入力はエルミート行列である必要がある⇒下三角行列で固有値を算出するため）
eVal, eVec = np.linalg.eigh(cov)

In [23]:
# 降順ソートのインデックス順列
indices = eVal.argsort()[::-1]

In [24]:
# 固有値降順で並び替え
eVal, eVec = eVal[indices], eVec[:, indices]

In [29]:
# デフォルトのリスク分布を設定（固有値最低の主成分に全て配分）
riskDist = np.zeros(cov.shape[0])
riskDist[-1] = 1

In [37]:
# 設定したリスク分布となる資産ウェイトの算出
loads = riskTarget*riskDist/eVal**.5
wghts = np.dot(eVec, np.reshape(loads, (-1, 1)))

In [9]:
prc = pd_data["Close"]["IBM"].sort_index(ascending=True)

In [12]:
diff = prc.diff()